In [1]:
import os
import shutil
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

import tensorflow as tf
from tensorflow.keras import backend as keras_backend
import numpy as np

gpu_on = True

if gpu_on :
    gpu_devices = tf.config.experimental.list_physical_devices("GPU")
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    gpu_devices = tf.config.experimental.list_physical_devices("CPU")

print(gpu_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
from src.data_loader.classification import ClassifyDataloader
from tensorflow.keras.applications.inception_v3 import preprocess_input
from glob import glob

task = "classification"
data_set_name = "detect_lvi"
batch_size = 16
on_memory = False
argumentation_proba = 0.8
target_size = (512,512)
interpolation = "bilinear"
class_mode = "binary"
# class_mode = "categorical"
dtype="float32"

train_image_path_list = glob(f"./datasets/{task}/{data_set_name}/train/*/*")
valid_image_path_list = glob(f"./datasets/{task}/{data_set_name}/valid/*/*")
test_image_path_list = glob(f"./datasets/{task}/{data_set_name}/test/*/*")
label_list = os.listdir(f"./datasets/{task}/{data_set_name}/train")

label_to_index_dict = {label:index for index, label in enumerate(label_list)}
index_to_label_dict = {index:label for index, label in enumerate(label_list)}

train_data_loader = ClassifyDataloader(image_path_list=train_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=batch_size,
                                       on_memory=on_memory,
                                       argumentation_proba=argumentation_proba,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,
                                       shuffle=True,
                                       class_mode=class_mode,
                                       dtype=dtype
)
valid_data_loader = ClassifyDataloader(image_path_list=valid_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=batch_size,
                                       on_memory=on_memory,
                                       argumentation_proba=0,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,                                       
                                       shuffle=False,
                                       class_mode=class_mode,
                                       dtype=dtype
)
test_data_loader = ClassifyDataloader(image_path_list=test_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=1,
                                       on_memory=False,
                                       argumentation_proba=0,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,                                       
                                       shuffle=False,
                                       class_mode=class_mode,
                                       dtype=dtype
)

Total data num 8608 with 2 classes
Total data num 1609 with 2 classes
Total data num 1469 with 2 classes


In [3]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

import numpy as np
np.random.seed(1337)  # for reproducibility

DROPOUT_RATIO = 0.5

grad_cam = True
transfer_learning = False
epoch_release_frozen = 10
transfer_train_mode = "include_deep_layer"
layer_name_frozen_to = "mixed4"

#  binary_sigmoid, categorical_sigmoid, categorical_softmax
activation = "binary_sigmoid"

# create the base pre-trained model~
base_model = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    classes=None,
    pooling=None,
    classifier_activation=None
)

if transfer_learning:
    if transfer_train_mode == "dense_only":
        base_model.trainable = False
    elif transfer_train_mode == "include_deep_layer":
        for layer in base_model.layers: 
            layer.trainable = False
            if layer.name == layer_name_frozen_to:
                break

# add a global spatial average pooling layer
x = base_model.output
# (Batch_Size,?)
x = GlobalAveragePooling2D()(x)
x = Dropout(DROPOUT_RATIO)(x)
# let's add a fully-connected layer
# (Batch_Size,1)
x = Dense(1024, activation='relu')(x)
# (Batch_Size,1024)
x = Dropout(DROPOUT_RATIO)(x)

if grad_cam:
    x *= 1e-1
    keras_backend.set_floatx('float64')
    dense_dtype = "float64"
else:
    dense_dtype = "float32"
    
if activation == "binary_sigmoid":
    predictions = Dense(1, activation='sigmoid', dtype=dense_dtype)(x)
    loss_function = BinaryCrossentropy(label_smoothing=0.01)
elif activation == "categorical_sigmoid":
    predictions = Dense(2, activation='sigmoid', dtype=dense_dtype)(x)
    loss_function = CategoricalCrossentropy(label_smoothing=0.01)
elif activation == "categorical_softmax":
    predictions = Dense(2, activation='softmax', dtype=dense_dtype)(x)
    loss_function = CategoricalCrossentropy(label_smoothing=0.01)

# this is the model we will train
model = Model(base_model.input, predictions)

# imported_weight_path = "./weights/classification/LVI_detect_classfication/binary_sigmoid/weights_1.0_1.0_InceptionV3_Imagenet_LVI_level2/weights_129_0.0012.hdf5"
# model.load_weights(imported_weight_path)

In [4]:
from datetime import date

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import Nadam

today = date.today()

# YY/MM/dd
today_str = today.strftime("%Y-%m-%d")
today_weight_path = f"./weights/{task}/{data_set_name}/{today_str}/target_size_{target_size}/" 
today_logs_path = f"./logs/{task}/{data_set_name}/{today_str}/target_size_{target_size}/"
os.makedirs(today_weight_path, exist_ok=True)
os.makedirs(today_logs_path, exist_ok=True)
optimizer = Nadam(1e-3, clipnorm=1)

save_c = ModelCheckpoint(
    today_weight_path+"/weights_{val_loss:.4f}_{loss:.4f}_{epoch:02d}.hdf5",
    monitor='val_loss',
    verbose=0,
    save_best_only=False,
    save_weights_only=True,
    mode='min')


reduceLROnPlat = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=20,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=5,
    min_lr=1e-7)
csv_logger = CSVLogger(f'{today_logs_path}/log.csv', append=False, separator=',')


def make_model_trainable(epoch, model, target_epoch):
    if epoch > target_epoch:
        for layer in model.layers:
            layer.trainable = True
            
make_trainable_callback = LambdaCallback(
    on_epoch_begin=lambda epoch,logs: make_model_trainable(epoch, model, target_epoch=epoch_release_frozen)
)

model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

In [5]:
# start_epoch = 0
# epochs = 200

# model.fit(
#     train_data_loader,
#     validation_data=valid_data_loader,
#     epochs=epochs,
#     callbacks=[reduceLROnPlat, save_c, csv_logger],
#     initial_epoch=start_epoch
# )

# Class Activation Map

In [6]:
import cv2
import os
import numpy as np

def imread(img_path, channel=None):
    img_byte_stream = open(img_path.encode("utf-8"), "rb")
    img_byte_array = bytearray(img_byte_stream.read())
    img_numpy_array = np.asarray(img_byte_array, dtype=np.uint8)
    img_numpy_array = cv2.imdecode(
        img_numpy_array, cv2.IMREAD_UNCHANGED)
    if channel == "rgb":
        img_numpy_array = cv2.cvtColor(
            img_numpy_array, cv2.COLOR_BGR2RGB)
    if len(img_numpy_array.shape) == 2:
        img_numpy_array = np.expand_dims(img_numpy_array, axis=-1)
    return img_numpy_array

from tensorflow import keras
import matplotlib.cm as cm

def get_last_conv_name(model):
    layer_names = [layer.name for layer in model.layers]
    conv_layer_name = [layer_name for layer_name in layer_names if layer_name.find("conv") >= 0]
    last_conv_name = conv_layer_name[-1]
    
    return last_conv_name

def get_img_array(img_path, channel="rgb"):
    
    img_array = imread(img_path, channel="rgb")
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

def make_grad_model(model, target_layer_name):
    # Model input: model.input
    # Model output: [last_conv_layer_outputs, model.outputs]
    grad_model = keras.models.Model(
        model.input, [model.get_layer(target_layer_name).output, model.output]
    )
    return grad_model

def make_gradcam_heatmap(img_array, grad_model, pred_index=None):
    # compute
    # 1. last_conv_layer_output
    # 2. class_channel value in prediction
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # compute gradient 
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # compute gradient channel mean 
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # compute matmul(last_conv_layer_output, pooled_grads)
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def save_and_display_gradcam(img_array_normalized, heatmap, cam_path="cam.jpg", alpha=0.4, display_cam=True):
    # Load the original image
    img_array = (img_array_normalized + 1) * 127.5
    img_array = img_array.astype('uint8')
    img_array = img_array[0]
    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img_array.shape[1], img_array.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img_array
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    if display_cam:
        display(Image.open(cam_path))


def decode_classify_predictions(pred_array, index_to_label_dict):
    predicted_index = np.sum(np.argmax(preds, axis=1))
    predicted = index_to_label_dict[predicted_index]
    
    return predicted

def decode_binary_classify_predictions(pred_array):
    predicted = np.round(pred_array).astype("int32")
    predicted = predicted.flatten()[0]
    return predicted

def restore_sigmoid_value(y, decrease_ratio=1e1):
    original_x = np.log(y/(1-y))
    restored_x = decrease_ratio*original_x
    new_y = 1/(1+np.exp(-restored_x))
    return new_y


In [7]:
def remove_folder_png_files(folder_path):
    regxp_files = os.path.join(folder_path, "*.png")
    target_files = glob(regxp_files)
    for target_file in target_files:
        os.remove(target_file)

In [8]:
layer_names = [layer.name for layer in model.layers]
conv_layer_name = [layer_name for layer_name in layer_names if layer_name.find("conv") >= 0]
last_conv_name = conv_layer_name[-1]

print(last_conv_name)

conv2d_93


In [9]:
best_path = f"./result_daily/classification/detect_lvi/2021-08-04/2/target_size_(512, 512)/epoch_28_loss_0.3532_dice_0.9075(augv3).hdf5"
model.load_weights(best_path)
print(best_path)

./result_daily/classification/detect_lvi/2021-08-04/2/target_size_(512, 512)/epoch_28_loss_0.3532_dice_0.9075(augv3).hdf5


In [10]:
target_data_loader = test_data_loader

target_data_loader.batch_size = 1

model_predicted = model.predict(target_data_loader)
ground_truth = np.array(list(target_data_loader.data_getter.class_dict.values()))

model.evaluate(target_data_loader)

1469/1469 [==============================] - 85s 56ms/step - loss: 0.4702 - accuracy: 0.9918


[0.47023020300271157, 0.9918311776718857]

In [36]:
negative_group_1 = np.where(model_predicted < 0.5)[0]
negative_group_2 = np.where(model_predicted != 0)[0]
negative_confirm = np.where(model_predicted == 0)[0]
negative_not_confirm = np.intersect1d(negative_group_1, negative_group_2)

positive_group_1 = np.where(model_predicted > 0.5)[0]
positive_group_2 = np.where(model_predicted != 1)[0]
postive_confirm = np.where(model_predicted == 1)[0]
postive_not_confirm = np.intersect1d(positive_group_1, positive_group_2)

ground_truth_negative = np.where(ground_truth == 0)[0]
ground_truth_positive = np.where(ground_truth == 1)[0]

true_negative = np.intersect1d(negative_group_1, ground_truth_negative)
true_positive = np.intersect1d(positive_group_1, ground_truth_positive)
false_negative = np.intersect1d(negative_group_1, ground_truth_positive)
false_positive = np.intersect1d(positive_group_1, ground_truth_negative)


In [37]:
print(len(negative_confirm))
print(len(negative_not_confirm))
print(len(postive_confirm))
print(len(postive_not_confirm))

0
777
0
692


In [38]:
print(len(negative_confirm))
print(len(negative_not_confirm))
print(len(postive_confirm))
print(len(postive_not_confirm))

0
777
0
692


In [39]:
print(false_negative)
print(false_positive)

[1054 1074 1389 1390 1391 1394 1395 1398 1400 1451]
[337 451]


In [40]:
from matplotlib import pyplot as plt

target_layer_name = "mixed10"
target_cam_folder = f"./gradcam/{task}/{data_set_name}/augv3/"
true_negative_folder = f"{target_cam_folder}/true_negative/"
true_positive_folder = f"{target_cam_folder}/true_positive/"
false_negative_folder = f"{target_cam_folder}/false_negative/"
false_positive_folder = f"{target_cam_folder}/false_positive/"

os.makedirs(true_negative_folder, exist_ok=True)
os.makedirs(true_positive_folder, exist_ok=True)
os.makedirs(false_negative_folder, exist_ok=True)
os.makedirs(false_positive_folder, exist_ok=True)

remove_folder_png_files(true_negative_folder)
remove_folder_png_files(true_positive_folder)
remove_folder_png_files(false_negative_folder)
remove_folder_png_files(false_positive_folder)

grad_model = make_grad_model(model, target_layer_name)

for index, (img_array_normalized, label) in enumerate(target_data_loader):
    
    # Print what the top predicted class is
    preds = model.predict(img_array_normalized)
    preds_value = preds.flatten()[0]
    preds_value = restore_sigmoid_value(preds_value, decrease_ratio=1e1)
    predicted_label = decode_binary_classify_predictions(preds)
    label = decode_binary_classify_predictions(label)
    pred_index = None
    # pred_index = label_to_index_dict[predicted_label]

    print(preds)
    print(preds_value)
    print(f"Predicted: {predicted_label}")

    # Generate class activation heatmap
    heatmap = make_gradcam_heatmap(img_array_normalized, grad_model, pred_index=pred_index)
    
    if predicted_label == 0 and label == 0:
        cam_path = f"{true_negative_folder}/cam_{index}_{preds_value:.4f}.png"
    if predicted_label == 1 and label == 1:
        cam_path = f"{true_positive_folder}/cam_{index}_{preds_value:.4f}.png"
    if predicted_label == 0 and label == 1:
        cam_path = f"{false_negative_folder}/cam_{index}_{preds_value:.4f}.png"
    if predicted_label == 1 and label == 0:
        cam_path = f"{false_positive_folder}/cam_{index}_{preds_value:.4f}.png"
    
    save_and_display_gradcam(img_array_normalized, heatmap, cam_path=cam_path, display_cam=False)
    

[[0.36851509]]
0.004559720635430568
Predicted: 0
[[0.369111]]
0.004677396649660837
Predicted: 0
[[0.37192315]]
0.005273607825496554
Predicted: 0
[[0.36805873]]
0.00447154539281278
Predicted: 0
[[0.36715951]]
0.00430261648130559
Predicted: 0
[[0.36715792]]
0.004302323303637693
Predicted: 0
[[0.3719659]]
0.005283217535399647
Predicted: 0
[[0.37046438]]
0.004955687811736655
Predicted: 0
[[0.35210001]]
0.0022418468692101006
Predicted: 0
[[0.37071983]]
0.005009986018402642
Predicted: 0
[[0.3672574]]
0.0043207025423896836
Predicted: 0
[[0.36854777]]
0.0045660988876753345
Predicted: 0
[[0.37286312]]
0.0054888401724126745
Predicted: 0
[[0.36829904]]
0.0045177687119021225
Predicted: 0
[[0.36935441]]
0.004726306418909925
Predicted: 0
[[0.3670991]]
0.004291491390064001
Predicted: 0
[[0.37408157]]
0.005780535022380271
Predicted: 0
[[0.36084549]]
0.003278898087877435
Predicted: 0
[[0.37058081]]
0.004980363823880685
Predicted: 0
[[0.3640865]]
0.003770722525876325
Predicted: 0
[[0.37335756]]
0.005605

[[0.37975268]]
0.00734778754071759
Predicted: 0
[[0.36595225]]
0.004085513157142285
Predicted: 0
[[0.37193522]]
0.005276320863819236
Predicted: 0
[[0.41941891]]
0.03726969669647231
Predicted: 0
[[0.36640023]]
0.0041648062507669064
Predicted: 0
[[0.36752481]]
0.0043704828516151545
Predicted: 0
[[0.38171963]]
0.007981968117126799
Predicted: 0
[[0.3755447]]
0.006150683916689276
Predicted: 0
[[0.37497993]]
0.0060051621466662135
Predicted: 0
[[0.3836309]]
0.008648777410847033
Predicted: 0
[[0.35948255]]
0.0030911766812256316
Predicted: 0
[[0.36583912]]
0.004065722000169273
Predicted: 0
[[0.41697974]]
0.033834859262769995
Predicted: 0
[[0.3681932]]
0.004497353572163728
Predicted: 0
[[0.36773984]]
0.00441091612573711
Predicted: 0
[[0.38279944]]
0.008352320237115641
Predicted: 0
[[0.3737102]]
0.005690072830990384
Predicted: 0
[[0.36576441]]
0.004052701864949176
Predicted: 0
[[0.36845802]]
0.004548603105082366
Predicted: 0
[[0.36633853]]
0.0041537972358544305
Predicted: 0
[[0.37004314]]
0.00486

[[0.36974954]]
0.004806757520992256
Predicted: 0
[[0.50431021]]
0.5429966672199211
Predicted: 1
[[0.38512076]]
0.009205650259824621
Predicted: 0
[[0.36821414]]
0.004501386978330932
Predicted: 0
[[0.36665816]]
0.004211132726052843
Predicted: 0
[[0.37254211]]
0.005414396324138842
Predicted: 0
[[0.36236206]]
0.0035007623092270885
Predicted: 0
[[0.37078445]]
0.005023810768759179
Predicted: 0
[[0.36448882]]
0.0038365495809489303
Predicted: 0
[[0.36846948]]
0.004550833126549262
Predicted: 0
[[0.37606581]]
0.006287981707693892
Predicted: 0
[[0.37585388]]
0.0062317881563024955
Predicted: 0
[[0.37083888]]
0.005035486067588449
Predicted: 0
[[0.36746109]]
0.004358570693676254
Predicted: 0
[[0.36118477]]
0.0033273172929340084
Predicted: 0
[[0.36809883]]
0.00447922759088311
Predicted: 0
[[0.3962413]]
0.014607393392530558
Predicted: 0
[[0.36664846]]
0.004209381539268942
Predicted: 0
[[0.36882081]]
0.004619729506577015
Predicted: 0
[[0.37193987]]
0.0052773656021733945
Predicted: 0
[[0.36749303]]
0.00

[[0.38796637]]
0.010367212476742695
Predicted: 0
[[0.37039473]]
0.004940983067117716
Predicted: 0
[[0.36997092]]
0.004852409931804391
Predicted: 0
[[0.36506411]]
0.003932614812494665
Predicted: 0
[[0.36869904]]
0.004595736412822318
Predicted: 0
[[0.37104294]]
0.005079490939025071
Predicted: 0
[[0.37080832]]
0.005028927696819698
Predicted: 0
[[0.36977462]]
0.00481190810239035
Predicted: 0
[[0.3730951]]
0.005543253341268843
Predicted: 0
[[0.36775581]]
0.00441393305789478
Predicted: 0
[[0.37084073]]
0.005035882975552
Predicted: 0
[[0.37029997]]
0.004921043731562566
Predicted: 0
[[0.36784411]]
0.004430652874258141
Predicted: 0
[[0.36940007]]
0.004735534610238462
Predicted: 0
[[0.41027655]]
0.025876007845664264
Predicted: 0
[[0.37481606]]
0.005963567295533223
Predicted: 0
[[0.36728964]]
0.004326674157914607
Predicted: 0
[[0.36993389]]
0.00484474385519309
Predicted: 0
[[0.36862565]]
0.004581334410789675
Predicted: 0
[[0.36715962]]
0.004302637062210147
Predicted: 0
[[0.36498639]]
0.0039195009

[[0.38367182]]
0.008663626915745942
Predicted: 0
[[0.37462155]]
0.005914557034018363
Predicted: 0
[[0.36971759]]
0.004800201474170093
Predicted: 0
[[0.36580941]]
0.004060540150899825
Predicted: 0
[[0.36693536]]
0.004261479130802875
Predicted: 0
[[0.37231764]]
0.005362922572671982
Predicted: 0
[[0.3685102]]
0.004558765653420958
Predicted: 0
[[0.35949977]]
0.0030934832555029705
Predicted: 0
[[0.37043631]]
0.004949756879298687
Predicted: 0
[[0.3604305]]
0.003220601162540723
Predicted: 0
[[0.36910508]]
0.004676213587638951
Predicted: 0
[[0.35641879]]
0.0027064211516035224
Predicted: 0
[[0.3645595]]
0.0038482285334081327
Predicted: 0
[[0.37013059]]
0.00488559645419374
Predicted: 0
[[0.36639415]]
0.004163720059921598
Predicted: 0
[[0.36834119]]
0.004525924378574136
Predicted: 0
[[0.36860254]]
0.004576808078960729
Predicted: 0
[[0.36485883]]
0.0038980711171543716
Predicted: 0
[[0.37101612]]
0.0050736858660441735
Predicted: 0
[[0.36951887]]
0.004759631409745701
Predicted: 0
[[0.36070618]]
0.00

[[0.62976348]]
0.9950922648332244
Predicted: 1
[[0.63064122]]
0.9952728129008377
Predicted: 1
[[0.62814346]]
0.9947413285552548
Predicted: 1
[[0.6314492]]
0.9954333096452765
Predicted: 1
[[0.63111815]]
0.995368198922448
Predicted: 1
[[0.62699269]]
0.994477399422024
Predicted: 1
[[0.6304367]]
0.9952313276465963
Predicted: 1
[[0.63529175]]
0.9961270807816504
Predicted: 1
[[0.63578045]]
0.9962076293401111
Predicted: 1
[[0.62934228]]
0.9950032704512999
Predicted: 1
[[0.63051878]]
0.9952480187290265
Predicted: 1
[[0.63267311]]
0.9956664156389339
Predicted: 1
[[0.63143579]]
0.9954306891159669
Predicted: 1
[[0.63242767]]
0.9956206121755976
Predicted: 1
[[0.6289266]]
0.9949139085704549
Predicted: 1
[[0.63204534]]
0.9955483284049571
Predicted: 1
[[0.63257931]]
0.9956489662051659
Predicted: 1
[[0.6330712]]
0.9957397179685343
Predicted: 1
[[0.62883825]]
0.9948947176347076
Predicted: 1
[[0.63361108]]
0.9958372153218843
Predicted: 1
[[0.63248016]]
0.9956304466752607
Predicted: 1
[[0.62685449]]
0.99

[[0.63315534]]
0.9957550571120035
Predicted: 1
[[0.63464687]]
0.996018248751829
Predicted: 1
[[0.63284596]]
0.9956983918869529
Predicted: 1
[[0.63201394]]
0.9955423408151465
Predicted: 1
[[0.63282452]]
0.9956944392795188
Predicted: 1
[[0.63385172]]
0.9958799705924232
Predicted: 1
[[0.63243699]]
0.9956223605962985
Predicted: 1
[[0.62726161]]
0.9945402017468793
Predicted: 1
[[0.63988602]]
0.9968231864634298
Predicted: 1
[[0.62855214]]
0.9948320860821634
Predicted: 1
[[0.61094593]]
0.9891523661219085
Predicted: 1
[[0.61600533]]
0.9912183405878293
Predicted: 1
[[0.58636982]]
0.9703956950088609
Predicted: 1
[[0.52153245]]
0.7030431089838151
Predicted: 1
[[0.53766495]]
0.8189807879048989
Predicted: 1
[[0.60497021]]
0.9861042705285828
Predicted: 1
[[0.60084134]]
0.9835318103329399
Predicted: 1
[[0.59545154]]
0.9794772428093981
Predicted: 1
[[0.62587558]]
0.994208939487383
Predicted: 1
[[0.60544067]]
0.9863715060727253
Predicted: 1
[[0.57144252]]
0.9467173770683046
Predicted: 1
[[0.58534973]]


[[0.59212837]]
0.9765168653025382
Predicted: 1
[[0.58328911]]
0.9665248317987037
Predicted: 1
[[0.63711545]]
0.9964194845317959
Predicted: 1
[[0.59135082]]
0.9757673698928451
Predicted: 1
[[0.61945061]]
0.9924014470814859
Predicted: 1
[[0.61279293]]
0.9899558950094752
Predicted: 1
[[0.50434949]]
0.5433865739697792
Predicted: 1
[[0.59961182]]
0.9826809887454039
Predicted: 1
[[0.5990206]]
0.9822569521759669
Predicted: 1
[[0.62746723]]
0.9945877552201587
Predicted: 1
[[0.61843284]]
0.9920690295374172
Predicted: 1
[[0.61002158]]
0.9887272384361596
Predicted: 1
[[0.6006635]]
0.9834112790824396
Predicted: 1
[[0.63235134]]
0.9956062727592255
Predicted: 1
[[0.6326332]]
0.9956589996105306
Predicted: 1
[[0.63492807]]
0.9960660634263645
Predicted: 1
[[0.6332036]]
0.9957638312183104
Predicted: 1
[[0.63422277]]
0.9959450668370221
Predicted: 1
[[0.63313082]]
0.9957505933506592
Predicted: 1
[[0.63003684]]
0.9951491973964484
Predicted: 1
[[0.63736105]]
0.9964571900024983
Predicted: 1
[[0.63312064]]
0.

[[0.63223939]]
0.995585160245848
Predicted: 1
[[0.63582477]]
0.9962148546613606
Predicted: 1
[[0.63254553]]
0.9956426648071902
Predicted: 1
[[0.63218293]]
0.9955744736722407
Predicted: 1
[[0.63638537]]
0.9963050889086639
Predicted: 1
[[0.62892786]]
0.994914183050829
Predicted: 1
[[0.63255056]]
0.9956436045592925
Predicted: 1
[[0.63286646]]
0.9957021689463137
Predicted: 1
[[0.63267488]]
0.9956667436447668
Predicted: 1
[[0.63520046]]
0.9961118517960427
Predicted: 1
[[0.63332494]]
0.9957858142180082
Predicted: 1
[[0.63438022]]
0.9959723871316634
Predicted: 1
[[0.63204838]]
0.9955489083379458
Predicted: 1
[[0.63515697]]
0.9961045747894325
Predicted: 1
[[0.63619711]]
0.9962750220609787
Predicted: 1
[[0.62985895]]
0.9951122220523966
Predicted: 1
[[0.63279357]]
0.9956887242083308
Predicted: 1
[[0.6316979]]
0.9954816387694808
Predicted: 1
[[0.63421758]]
0.9959441629636012
Predicted: 1
[[0.37240759]]
0.005383490933749027
Predicted: 0
[[0.49525896]]
0.452729807401846
Predicted: 0
[[0.49500311]]
